# TODO:

- Stitch
- 4B compatibility

In [1]:
import os
import numpy as np
import scipy.stats
import plotly.offline as py
import plotly.graph_objs as go
py.init_notebook_mode(connected=True)

import warnings
warnings.filterwarnings('ignore', module='numpy')
warnings.filterwarnings('ignore')

In [2]:
import sys
sys.path.append("/SNS/users/m2d/git/reflectivity_ui/test/notebooks")
from plot_utils import plot1d, plot_heatmap
import mantid
import mantid.simpleapi as api

In [293]:
import event_reduction
reload(event_reduction)
from event_reduction import load_data, get_wl_range, get_peak, quicknxs_scale, get_q_binning, EventReflectivity
import datasets
reload(datasets)

<module 'datasets' from '/SNS/users/m2d/git/reflectivity_ui/test/notebooks/datasets.pyc'>

In [274]:
#data_id = '30891'
#data_id = '30806'
data_id = '30889'
#data_id = '30906'
data_sc, data_db = datasets.retrieve(data_id)

peak_center = data_sc['peak_pos']
peak = data_sc['peak']
peak_bck = data_sc['bck']
beam = data_sc['beam']
tof = data_sc['tof']

norm_peak = data_db['peak']
norm_bck = data_db['bck']
norm_beam = data_db['beam']

ref_path = data_sc['ref']
ref_cst_path = data_sc['refcst']
ref_scale = data_sc['scale']

In [275]:
%%time
ws_sc = load_data(run=data_sc['sc'])
ws_db = load_data(run=data_db['sc'])

CPU times: user 6.52 s, sys: 1.14 s, total: 7.65 s
Wall time: 5.86 s


In [294]:
%%time

theta = ws_sc.getRun()['SANGLE'].getStatistics().mean * np.pi / 180.
#theta = 0.0110254156154/2.0

event_refl = EventReflectivity(ws_sc, ws_db,
                               signal_peak=peak, signal_bck=peak_bck,
                               norm_peak=norm_peak, norm_bck=norm_bck,
                               specular_pixel=peak_center,
                               signal_low_res=beam, norm_low_res=norm_beam,
                               q_min=None, q_step=-0.02, q_max=None,
                               #tof_range=tof,
                               theta=theta)

print(event_refl)
_scale = quicknxs_scale(theta, peak, beam, norm_peak, norm_beam)
print("Quicknxs scale = %s" % _scale)
print("Reference scale = %s" % ref_scale)

sample-det: 2.297
pixel: 0.0007
WL: 2.75 7.95
Q: 0.0118944200218 0.034385686972
Theta = 0.00752496759002
Quicknxs scale = 0.875646226198
Reference scale = 2.8379
CPU times: user 0 ns, sys: 1.81 ms, total: 1.81 ms
Wall time: 1.69 ms


## Compute event-based reflectivity

Old Mantid algorithm takes 1.3 sec to compute reflectivity

In [277]:
%%time

q_bins, evt_r, evt_dr = event_refl.specular()

# Scale to compare to QuickNXS results
refl = evt_r * _scale * ref_scale
d_refl = evt_dr * _scale * ref_scale

CPU times: user 1.62 s, sys: 137 ms, total: 1.76 s
Wall time: 1.72 s


In [300]:
%%time
# EventReflectivity.DELTA_KZ_VS_QZ
# EventReflectivity.KZI_VS_KZF


if 0:
    #qx_bins, qz_bins, _refl_2d, _d_refl_2d = event_refl.off_specular(x_min=-0.00003, x_max=0.000035, x_npts=25,
    #                                                             z_min=-0.001, z_max=0.035, z_npts=100,
    qx_bins, qz_bins, _refl_2d, _d_refl_2d = event_refl.off_specular(x_min=-0.0002, x_max=0.0001, x_npts=35,
                                                                 z_min=-0.001, z_max=0.06, z_npts=100,
                                                                 bck_in_q=[9e-5, 22e-5],
                                                                 x_axis=None)
elif 1:
    qx_bins, qz_bins, _refl_2d, _d_refl_2d = event_refl.off_specular(x_min=-0.005, x_max=0.022, x_npts=60,
                                                                 z_min=-0.001, z_max=0.06, z_npts=50,
                                                                 bck_in_q=[0.002,0.004],
                                                                 x_axis=EventReflectivity.DELTA_KZ_VS_QZ )

elif 1:
    qx_bins, qz_bins, _refl_2d, _d_refl_2d = event_refl.off_specular(z_min=-0.01, z_max=0.015, z_npts=50,
                                                                 x_min=0.004, x_max=0.015, x_npts=50,
                                                                 bck_in_q=None,
                                                                 x_axis=EventReflectivity.KZI_VS_KZF )

#qx_bins, qz_bins, _refl_2d, _d_refl_2d = event_refl.off_specular(bck_in_q=[0.002, 0.004])
#qx_bins, qz_bins, _refl_2d, _d_refl_2d = event_refl.off_specular(bck_in_q=[-0.0018, 0.0018])

refl_2d = _refl_2d * _scale * ref_scale
d_refl_2d = _d_refl_2d * _scale * ref_scale

CPU times: user 24.7 s, sys: 43.2 s, total: 1min 7s
Wall time: 37.5 s


In [302]:
print ref_cst_path
print ref_path
spec_data_cst = np.loadtxt(ref_cst_path)
spec_data = np.loadtxt(ref_path)


q_bins = event_refl.q_bins
q_middle = [(q_bins[i+1]+q_bins[i])/2.0 for i in range(len(q_bins)-1)]

_plot_list = [[q_middle, refl, d_refl],
              [spec_data.T[0], spec_data.T[1], spec_data.T[2]],
              [spec_data_cst.T[0], spec_data_cst.T[1], spec_data_cst.T[2]],
             ]
_plot_names = ["Events 1d", "Old", "Old cst-Q"]

if True:
    # Qx range for the specular ridge in 2D
    _z, _spec, _d_spec = event_refl.slice(-0.0016, 0.0016, normalize=False)
    #_z, _spec, _d_spec = event_refl.slice(-1.3e-5, 1.3e-5, normalize=False)

    _sum_scale = 5.0

    _spec *= _scale * ref_scale * _sum_scale
    _d_spec *= _scale * ref_scale * _sum_scale

    _z_middle = [(_z[i+1]+_z[i])/2.0 for i in range(len(_z)-1)]

    _plot_list.append([_z_middle, _spec, _d_spec])
    _plot_names.append("Events 2D")

    

plot1d(_plot_list, _plot_names, x_log=True, y_log=True, x_title='Q')

/SNS/users/m2d/MR/REF_M_30889+30890_Specular_Off_Off_CSTQ.dat
/SNS/users/m2d/MR/REF_M_30889+30890_Specular_Off_Off.dat


In [301]:
plot_heatmap(qx_bins, qz_bins, np.log(refl_2d).T)

## Reference: DON'T TOUCH

In [675]:
plot_heatmap(qx_bins, qz_bins, np.log(refl_2d).T)

In [16]:
plot_heatmap(qx_bins, qz_bins, np.log(refl_2d).T)